In [1]:
import sqlite3
import pandas as pd
db = sqlite3.connect("flights.db")

In [4]:
pd.read_sql("""
SELECT *
FROM airports;
""", db)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
8103,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
8104,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
8105,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


In [5]:
pd.read_sql("""
SELECT *
FROM airlines;
""", db)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
...,...,...,...,...,...,...,...,...,...
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y


In [6]:
pd.read_sql("""
SELECT *
FROM routes;
""", db)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


# Subqueries in `FROM`
This is useful if you want to apply multiple aggregation functions

E.g What is the average number of routes departing from all airports?

**Steps:**
1. Get the total number of routes departing from all airports
2. Take the average

In [8]:
pd.read_sql("""
SELECT 
    source,
    COUNT() AS num_departures
FROM routes
GROUP BY source;
""", db)
# This is now our subquery

,source,num_departures
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


In [9]:
pd.read_sql("""
SELECT AVG(num_departures)
FROM (
    SELECT 
    source,
    COUNT() AS num_departures
    FROM routes
    GROUP BY source
)
""", db)

,AVG(num_departures)
0,19.848343


# SUBQUERIES IN `WHERE`

This is useful when you want to filter a query based on the results from another query

E.g. What are the departures and destinations for flight routes from the top 5 countries with the most airports.

**Steps**
1. Identify the country with the most airports

In [13]:
pd.read_sql("""
SELECT country, COUNT() AS num_airports
FROM airports
GROUP BY country
ORDER BY num_airports DESC
LIMIT 5;
""", db)

,country,num_airports
0,United States,1697
1,Canada,435
2,Germany,321
3,Australia,263
4,Russia,249


2. Enter this query into a new query of the routes table get the data needed

In [16]:
pd.read_sql("""
SELECT rt.source, rt.dest, ap.country
FROM routes AS rt
LEFT JOIN airports AS ap
    ON rt.source_id = ap.id
WHERE ap.country IN (
    SELECT country
    FROM airports
    GROUP BY country
    ORDER BY COUNT() DESC
    LIMIT 5
)
ORDER BY ap.country;
""", db)

,source,dest,country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


# CTEs

Common Table Expressions are a more readable way to implement subqueries using `WITH` and `AS`

In [18]:
pd.read_sql("""
WITH top_5_countries AS
    (
    SELECT country
    FROM airports
    GROUP BY country
    ORDER BY COUNT() DESC
    LIMIT 5
    )
SELECT
    source, dest, country
FROM routes
JOIN airports
    ON routes.source_id = airports.id
WHERE country IN top_5_countries
ORDER BY country;
""", db)

,source,dest,country
0,ABM,CNS,Australia
1,ASP,ADL,Australia
2,ASP,AYQ,Australia
3,ASP,BNE,Australia
4,ASP,CNS,Australia
...,...,...,...
20330,TNK,WWT,United States
20331,TNK,WWT,United States
20332,GNU,PTU,United States
20333,WWT,BET,United States


Create a table listing all airlines that serve the three airports with the most outbound routes

In [35]:
# Which airports have the three most outbound routes
pd.read_sql("""
SELECT  ap.code
FROM airports AS ap
LEFT JOIN routes AS r
    ON r.source_id = ap.id
GROUP BY ap.name
ORDER BY COUNT(*) DESC
LIMIT 3;
""", db)

,code
0,ATL
1,ORD
2,PEK


In [39]:
# My Attempt
pd.read_sql("""
WITH top_3_airports_with_most_destinations AS (
    SELECT ap.code
    FROM airports AS ap
    LEFT JOIN routes AS r
        ON r.source_id = ap.id
    GROUP BY ap.name
    ORDER BY COUNT(*) DESC
    LIMIT 3
)

SELECT DISTINCT airline
FROM airlines AS al
JOIN routes AS r
    ON al.id = r.airline_id
WHERE dest in top_3_airports_with_most_destinations;
""", db)

,airline
0,3E
1,3U
2,5J
3,8L
4,9E
...,...
95,WN
96,WS
97,Y4
98,Y7


In [42]:
# Greg's Attempt
pd.read_sql("""
WITH top_3_airports AS (
    SELECT airports.id
    FROM airports
    LEFT JOIN routes
        ON routes.source_id = airports.id
    GROUP BY airports.id
    ORDER BY COUNT() DESC
    LIMIT 3
)
SELECT DISTINCT airline
FROM routes
LEFT JOIN airports
    ON routes.source_id = airports.id
WHERE routes.source_id IN top_3_airports
""", db)

,airline
0,3E
1,3M
2,3U
3,5J
4,8L
...,...
97,WN
98,WS
99,Y4
100,Y7
